# Algae Temperature Tolerance

## Setup

Analysis and visualization was done using R and various packages.

In [1]:
library(tidyverse)
library(ggplot2)
library(cowplot)
options(repr.plot.width=10, repr.plot.height=6,warn=-1)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'cowplot' was built under R version 3.6.3"

## Reading and Wrangling Data

The entire dataset is found at `"data/GlobalTherm_upload_02_11_17.csv"`.

In [2]:
global_term_data <- read_csv("data/GlobalTherm_upload_02_11_17.csv") %>%
    select(Genus, Species, N, Tmax, lat_max, long_max, tmin)

Parsed with column specification:
cols(
  .default = col_double(),
  Genus = col_character(),
  Species = col_character(),
  max_metric = col_character(),
  `error measure` = col_character(),
  `Multiple measures` = col_character(),
  max_metric_2 = col_character(),
  error_1 = col_logical(),
  `error measure_1` = col_logical(),
  max_pretreatment = col_character(),
  REF_max = col_character(),
  min_metric = col_character(),
  error_2 = col_character(),
  error_type = col_character(),
  `Multiple measures_min` = col_character(),
  N_2 = col_logical(),
  min_metric_2 = col_character(),
  error_3 = col_logical(),
  error_type_1 = col_logical(),
  min_pretreatment = col_character(),
  REF_min = col_character()
  # ... with 4 more columns
)
See spec(...) for full column specifications.


In [3]:
head(global_term_data)

Genus,Species,N,Tmax,lat_max,long_max,tmin
Prochilodus,scrofa,10,35.7,-22.0,-43.50,8.23
Lepomis,gibbosus,24,35.1,46.2,-119.00,1.70
Cyprinodon,variegatus,100,44.2,26.0,-97.15,6.90
Oncorhynchus,keta,NA,23.2,52.0,-128.90,-0.70
Oncorhynchus,nerka,NA,21.5,52.0,-128.90,-0.70
Oncorhynchus,kisutch,60,27.5,49.4,-123.40,0.00


Let's load in the ocean temp data as well. We will need to see for what range of latitude and longitude we will need to filter for. This data was processed and created in the [](./Ocean_Temperature.ipynb) chapter.

In [4]:
temperature_data <- read_csv("data/nav_temp_joined_processed.csv")

Parsed with column specification:
cols(
  datetime = col_datetime(format = ""),
  mean_ext = col_double(),
  mean_lat = col_double(),
  mean_long = col_double(),
  mean_deg_lat = col_double(),
  mean_deg_long = col_double()
)


In [5]:
summary(select(temperature_data,mean_deg_lat, mean_deg_long, mean_ext ))

  mean_deg_lat   mean_deg_long      mean_ext     
 Min.   :31.54   Min.   :117.2   Min.   : 9.618  
 1st Qu.:33.74   1st Qu.:120.6   1st Qu.:13.096  
 Median :37.83   Median :123.4   Median :14.771  
 Mean   :39.41   Mean   :123.1   Mean   :15.165  
 3rd Qu.:45.25   3rd Qu.:125.0   3rd Qu.:16.586  
 Max.   :52.64   Max.   :130.8   Max.   :24.086  

If we round the latitude & longitude by `round_down_min - 1`, and `round_up_max + 1`:

- The range for latitude can be summarized as 30 ~ 54. 
- The range for longitude can be summarized as 116 ~ 132.

Then, we can ask for all of the unique genus found within this range.

In [6]:
filter(global_term_data, between(lat_max, 30, 54), between(long_max, 116, 132)) %>%
    distinct(Genus)

Genus
Plestiodon
Sphenomorphus
Eremias
Takydromus
Apodemus
Myodes
Cricetulus
Meriones


Unfortunately, the list of genus does not contain any algae. 

By expanding the range, however...

In [7]:
filter(global_term_data, between(lat_max, 20, 60), between(long_max, 120, 140)) %>%
    distinct(Genus)

Genus
Engraulis
Sardinops
Acridotheres
Pycnonotus
Chondracanthus
Paratrechina
Pheidole
Pristomyrmex
Technomyrmex
Tetramorium


We gain these three algae genuses.
- [Sargassum](https://www.britannica.com/science/Sargassum)
- [myagropsis](http://www.marinespecies.org/aphia.php?p=taxdetails&id=372020)
- [Chondracanthus](https://www.algaebase.org/search/species/detail/?species_id=127)

In [8]:
algae_west_coast <- filter(global_term_data, 
       Genus %in% c("Sargassum", "myagropsis", "Chondracanthus"),
      long_max > 0) %>%
    select(Genus, Species, Tmax)

algae_west_coast

Genus,Species,Tmax
Chondracanthus,intermedius,31.3
myagropsis,Myagroides,30.0
Sargassum,fulvellum,30.0
Sargassum,hemiphyllum,27.0
Sargassum,horneri,27.0
Sargassum,macrocarpum,31.0
Sargassum,micracanthum,27.0
Sargassum,patens,31.0
Sargassum,piluliferum,30.0
Sargassum,thunbergii,31.0


With a quick glance at this table, we can see that the max mean of min temperature during the heatwave (21.71) never exceeded the max temperatures of these species.

In [9]:
head(temperature_data)

datetime,mean_ext,mean_lat,mean_long,mean_deg_lat,mean_deg_long
2021-06-13 00:00:20,19.05565,32.40404,117.1328,32.67333,117.2213
2021-06-13 00:00:21,18.62864,32.39704,117.1738,32.66167,117.2897
2021-06-13 00:00:22,19.00354,32.39495,117.1988,32.65833,117.3313
2021-06-13 00:00:23,19.09503,32.39151,117.2244,32.65250,117.3740
2021-06-13 00:00:24,19.17977,32.39131,117.2250,32.65217,117.3750
2021-06-13 00:00:25,19.10705,32.39111,117.2255,32.65183,117.3758


## Other Studies of Algae Species near BC and the Pacific Coast

### *Sargassum horneri*

To further support *Sargassum horneri*'s temperature tolerance, [a more recent study by Sadie L. Small and Matthew S. Edwards in 2021](https://doi.org/10.1111/jpy.13148) found that total mortality ranges around 28–30°C, with 15–20°C to be the optimal temperature range for germling development (germling development maxing out around 25°C).

### Dinoflagellates of genus *Alexandrium* in the Salish Sea

<sub>Also known as <i>A. tamarense</i> or <i>A. fundyense</i> or <i>A. catenella</i>.</sub>

In a [2016 study by Brian D. Bill et. al.](https://doi.org/10.1111/jpy.12386) referenced Noris and Chew's work in 1975 about how "optimal temperature and salinity ranges for growth were between 13°C and 17°C temperature and 15–40 salinity". The researchers have also found that "growth of both isolates was inhibited at temperatures below ~7.7°C and above ~24°C" where being above ~24°C resulted in "growth rates dropped abruptly to near zero" in some strains.

### *Phaeodactylum tricornutum*

Another study, [conducted in 2011 by Gudrun Kräbs and Claudia Büchel](https://doi.org/10.1515/bot.2011.037), focused on the eco-physiology of *Phaeodactylum tricornutum* and looked at it's three strains isolated in Vancouver (NE Pacific Ocean), Plymouth (NE Atlantic Ocean) and Guam (tropical western Pacific Ocean). 
> "The Vancouver and Plymouth strains are from cold-temperate conditions where water-temperature is between 10°C and in winter and 15°C in the summer" (p. 232). "Cultures of each strain were exposed to a combination of temperatures ranging from 12°C to 30°C."

At 24°C, the Vancouver strain started to see some vitality declines, with 30°C straining it to it's limits while the other strains tolerated, albeit not ideal conditions (p. 234-235). 

### *Calliarthron tuberculosum & Corallina vancouveriensis*

[This study by Rebecca J. Guenther and Patrick T. Martone in 2013](https://doi.org/10.1111/jpy.12161) looked at aerial exposure during low tide in tidepools, and the warming of water during which the coralline algae (*Calliarthron tuberculosum & Corallina vancouveriensis*) would be submerged/exposed and found that:



Aerial Exposure                     | Warming Tidepool Water (submerged)
------------------------------------|-----------------------
Calliarthron is extremely sensitive | Calliarthron is stressed
Corallina is somewhat stressed      | Corallina is not stressed

<center><sub>See section "Recovery when the tide returns" (Guenther & Martone, 2013) .</sub></center>

but they have also pointed out that:
> "Neither Corallina nor Calliarthron recovered photosynthesis after emersion as rapidly as other intertidal algae, such as *Mastocarpus papillatus*, *Endocladia muricata*, *Fucus serratus*, and *Fucus spiralis*".


<center><b>Photosynthetic rate after submersion:</b></center>

Species      | Photosynthetic rate (12°C) | Photosynthetic rate (20°C)
-------------|----------------------------|---------------------------
Calliarthron | 1.40 ± 0.29                | 1.13 ± 0.18
Corallina    | 1.77 ± 0.12                | 1.93 ± 0.07 

<center><sub><b>Photosynthetic rate units:</b> $\frac{\text{μmol of }O^2}{\text{(gDW · min)}}$</sub></center>

## Temperature Tolerance Studies on Predators

```{Attention}
This section is a work in progress.
```

[Potential predators](http://dx.doi.org/10.1002/ece3.6498) and their tolerance for ocean temperatures.
- Nucella lamellosa
- Littorina scutulata
- Littorina sitkana
- Balanus glandula